## COD API, Matches --> explore, format, clean & reshape

Activision Call of Duty API use case for **Matches** endpoint, using a slightly amended version of callofduty.py client and custom cleaning functions (from wzkd app)

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

#### Login using SSO

In [2]:
# We're storing our SSO token in an .env file stored locally to separate our conf from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### Slightly modify client methods to call the API matches endpoint

In [3]:
# This time we're adding additional methodsin the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use

# 1. Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# 2. following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# 3. add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

#### Get Matches data

In [ ]:
# In this notebook, we will only use the most interesting api's endpoint /...details, with the method Client.GetMatchesDetailed
# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("battle", "amadevs#1689", Title.ModernWarfare, Mode.Warzone, limit=20) #AMADEVS#1689

##### Option: save previous result so we're not getting annoyed by API rate limits or inconsistencies -,-

In [ ]:
with open("matches.pkl", 'wb') as f:
    pickle.dump(matches, f)

## Match result : structure

In [4]:
# load previously saved data
with open('matches.pkl', 'rb') as f:
    matches = pickle.load(f)

### Overview : dict --> df

In [5]:
df_matches = pd.DataFrame(matches)
display(df_matches.head(2))
keys = list(df_matches.keys())
keys.sort()
pprint(keys)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 9078...","{'team': 'team_twenty_seven', 'rank': 54.0, 'a...",75,None,False,False
1,1650401331,1650402980,mp_wz_island,br_brduos,9298769577395596626,1649000,None,1,wz,149,"{'kills': 2.0, 'medalXp': 90.0, 'matchXp': 666...","{'team': 'team_twelve', 'rank': 54.0, 'awards'...",75,None,False,False


['draw',
 'duration',
 'gameType',
 'map',
 'matchID',
 'mode',
 'player',
 'playerCount',
 'playerStats',
 'playlistName',
 'privateMatch',
 'rankedTeams',
 'teamCount',
 'utcEndSeconds',
 'utcStartSeconds',
 'version']


### API structure for one match (a 'row')

##### Reminder : matches endpoint returns n (max 20) matches & their associated stats, as 'rows', for a single --queried player

In [6]:
pprint(matches[10], depth=2)

{'draw': False,
 'duration': 1206000,
 'gameType': 'wz',
 'map': 'mp_escape4',
 'matchID': '13834318442993609455',
 'mode': 'br_dmz_playlist_wz325/rbrthbmo_quads',
 'player': {'awards': {...},
            'brMissionStats': {...},
            'clantag': 'IG.YT',
            'loadout': [...],
            'loadouts': [...],
            'rank': 54.0,
            'team': 'team_eight',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 37,
 'playerStats': {'assists': 2.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 7148.0,
                 'damageTaken': 1352.0,
                 'deaths': 8.0,
                 'distanceTraveled': 113326.56,
                 'executions': 0.0,
                 'headshots': 11.0,
                 'kdRatio': 3.5,
                 'kills': 28.0,
                 'longestStreak': 11.0,
                 'matchXp': 4962.0,
                 'medalXp': 1860.0,
  

### Focus : what's in 'playerStats' ?

In [7]:
player_stats = df_matches['playerStats'].apply(pd.Series)
display(player_stats.head(5))
pprint(player_stats.keys())

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,timePlayed,executions,gulagKills,nearmisses,percentTimeMoving,miscXp,longestStreak,teamPlacement,damageDone,damageTaken
0,0.0,0.0,9078.0,3900.0,0.0,2075.0,13004.0,0.0,1.0,0.0,...,1035.0,0.0,0.0,0.0,80.61117,0.0,0.0,17.0,105.0,737.0
1,2.0,90.0,6661.0,3375.0,0.0,2025.0,10310.0,2.0,0.0,0.0,...,754.0,0.0,1.0,0.0,77.57297,0.0,2.0,31.0,975.0,537.0
2,2.0,20.0,10407.0,5940.0,0.0,4275.0,16675.0,0.0,0.0,0.0,...,1229.0,0.0,0.0,0.0,68.37979,0.0,2.0,11.0,1010.0,1097.0
3,3.0,85.0,4130.0,3450.0,0.0,3250.0,7900.0,1.0,1.0,0.0,...,457.0,0.0,0.0,0.0,95.10870,0.0,3.0,47.0,957.0,617.0
4,0.0,30.0,3707.0,1675.0,0.0,1075.0,5444.0,0.0,1.0,0.0,...,460.0,0.0,0.0,0.0,88.77284,0.0,0.0,53.0,111.0,508.0


Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'percentTimeMoving', 'miscXp',
       'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken'],
      dtype='object')


In [8]:
displayed_cols = ['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']
# Just in case a specific game mode does not contain a value we wanted to display
displayed_cols = [col for col in displayed_cols if col in list(player_stats.keys())]
player_stats[displayed_cols].head(5)

,headshots,distanceTraveled,teamSurvivalTime
0,0.0,283549.40,950208.0
1,2.0,356449.12,650640.0
2,0.0,198202.19,1118784.0
3,1.0,344002.97,370848.0
4,0.0,166267.97,385488.0


### Focus : 'player', a nested entry

In [9]:
player = df_matches['player'].apply(pd.Series)
display(player.head(5))
pprint(player.keys())

,team,rank,awards,username,uno,clantag,loadouts,brMissionStats,loadout
0,team_twenty_seven,54.0,{},gentil_renard,2621859779580650696,IG.YT,"[{'primaryWeapon': {'name': 's4_sh_bromeo5', '...","{'missionsComplete': 1, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_sh_bromeo5', '..."
1,team_twelve,54.0,"{'low_health_kill': 285168.0, 'one_shot_kill':...",gentil_renard,2621859779580650696,IG.YT,"[{'primaryWeapon': {'name': 's4_sh_bromeo5', '...","{'missionsComplete': 2, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_sh_bromeo5', '..."
2,team_thirty_seven,54.0,"{'low_health_kill': 634800.0, 'mode_x_eliminat...",gentil_renard,2621859779580650696,IG.YT,"[{'primaryWeapon': {'name': 's4_sh_bromeo5', '...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_sh_bromeo5', '..."
3,team_thirty_one,54.0,"{'low_health_kill': 229968.0, 'gun_butt': 1038...",gentil_renard,2621859779580650696,IG.YT,"[{'primaryWeapon': {'name': 's4_sh_bromeo5', '...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_sh_bromeo5', '..."
4,team_forty,54.0,"{'revenge': 0.0, 'pointblank': 0.0, 'headshot'...",gentil_renard,2621859779580650696,IG.YT,"[{'primaryWeapon': {'name': 's4_sm_mpapa40', '...","{'missionsComplete': 1, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_sm_mpapa40', '..."


Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')


#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [10]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 2
pprint(player['loadout'][match_index][0], depth=2)

{'extraPerks': [{...}, {...}, {...}],
 'killstreaks': [{...}, {...}, {...}],
 'lethal': {'image': None,
            'imageLarge': None,
            'label': None,
            'name': 'equip_throwing_knife',
            'progressionImage': None},
 'perks': [{...}, {...}, {...}],
 'primaryWeapon': {'attachments': [...],
                   'imageIcon': None,
                   'imageLoot': None,
                   'label': None,
                   'name': 's4_sh_bromeo5',
                   'variant': '8'},
 'secondaryWeapon': {'attachments': [...],
                     'imageIcon': None,
                     'imageLoot': None,
                     'label': None,
                     'name': 'iw8_sn_t9accurate',
                     'variant': '0'},
 'tactical': {'image': None,
              'imageLarge': None,
              'label': None,
              'name': 'equip_adrenaline',
              'progressionImage': None}}


#### Inside 'player' entry, 'brMissionStats' is a (list of) dict

In [11]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 4
pprint(player['brMissionStats'][match_index], depth=3)

{'missionStatsByType': {'timedrun': {'count': 1.0,
                                     'weaponXp': 500.0,
                                     'xp': 500.0}},
 'missionsComplete': 1,
 'totalMissionWeaponXpEarned': 500.0,
 'totalMissionXpEarned': 500.0}


## Format & clean API **matches** result using customized tools (wzkd app)

In [12]:
import json
import toml
# functions defined in wzkd app directory '/wzkd/wzkd'
sys.path.insert(0, os.path.abspath('../wzkd'))
from utils import load_labels, load_conf
from api_format import res_to_df, format_df

In [13]:
# conf and labels files stored here as well.
# labels is needed for parsing games modes/weapons, conf stores values such as n of loadouts to extract or columns names
file_labels = "wz_labels.json"
filepath_labels = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_labels
LABELS = load_labels(filepath_labels)
pprint(LABELS, depth=2)

file_conf = "conf.toml"
filepath_conf = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/wzkd/" + file_conf
CONF = load_conf(filepath_conf)
pprint(CONF, depth=2)

{'modes': {'battle_royale': {...}, 'multiplayer': {...}},
 'weapons': {'cat_names': {...},
             'categories': [...],
             'names': {...},
             'prefixes': [...]}}
{'APP': {'br_only': True, 'mode': 'offline'},
 'FORMATTING': {'float_cols': [...], 'int_cols': [...], 'ts_cols': [...]},
 'PARSING': {'mission_types': [...], 'n_loadouts': 3},
 'RENDERING': {'keep_cols': {...}, 'labels': {...}}}


In [14]:
with open('matches.pkl', 'rb') as f:
    res = pickle.load(f)
tmp = pd.DataFrame(res)
display(tmp.head(2))
pprint(tmp.keys())

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,None,1,wz,156,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 9078...","{'team': 'team_twenty_seven', 'rank': 54.0, 'a...",75,None,False,False
1,1650401331,1650402980,mp_wz_island,br_brduos,9298769577395596626,1649000,None,1,wz,149,"{'kills': 2.0, 'medalXp': 90.0, 'matchXp': 666...","{'team': 'team_twelve', 'rank': 54.0, 'awards'...",75,None,False,False


Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'playlistName', 'version', 'gameType', 'playerCount',
       'playerStats', 'player', 'teamCount', 'rankedTeams', 'draw',
       'privateMatch'],
      dtype='object')


In [15]:
# flatten-expand into a DataFrame the result from COD API, for matches history
df_matches = res_to_df(res, CONF)
display(df_matches.head(5))
pprint(df_matches.keys())

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,domination,timedrun,assassination,masterassassination,scavenger
0,1650402208,1650403856,mp_wz_island,br_brduos,10717821121770145230,1648000,1,wz,156,75,...,"{'primaryWeapon': {'name': 's4_pi_mike1911', '...","{'primaryWeapon': {'name': 's4_sm_stango5', 'l...",1,650.0,650.0,"{'weaponXp': 650.0, 'xp': 650.0, 'count': 1.0}",NaN,NaN,NaN,NaN
1,1650401331,1650402980,mp_wz_island,br_brduos,9298769577395596626,1649000,1,wz,149,75,...,"{'primaryWeapon': {'name': 's4_pi_mike1911', '...",NaN,2,1075.0,1075.0,"{'weaponXp': 500.0, 'xp': 500.0, 'count': 1.0}","{'weaponXp': 575.0, 'xp': 575.0, 'count': 1.0}",NaN,NaN,NaN
2,1650400010,1650401603,mp_wz_island,br_brduos,4690918440001982609,1593000,1,wz,148,74,...,"{'primaryWeapon': {'name': 's4_pi_mike1911', '...","{'primaryWeapon': {'name': 's4_sm_stango5', 'l...",0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1650399438,1650401082,mp_wz_island,br_brduos,9316483071360501762,1644000,1,wz,151,75,...,"{'primaryWeapon': {'name': 's4_pi_mike1911', '...",NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,1650398890,1650400476,mp_wz_island,br_brduos,11643996262360176659,1586000,1,wz,153,76,...,NaN,NaN,1,500.0,500.0,NaN,"{'weaponXp': 500.0, 'xp': 500.0, 'count': 1.0}",NaN,NaN,NaN


Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'version', 'gameType', 'playerCount', 'teamCount', 'draw',
       'privateMatch', 'kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs',
       'score', 'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'percentTimeMoving', 'miscXp',
       'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken', 'team',
       'awards', 'username', 'uno', 'clantag', 'loadout_1', 'loadout_2',
       'loadout_3', 'missionsComplete', 'totalMissionXpEarned',
       'totalMissionWeaponXpEarned', 'domination', 'timedrun', 'assassination',
       'masterassassination', 'scavenger'],
      dtype='object')


In [16]:
# make the stats human-readable and parse some values (weapons, games modes)
df_formatted = format_df(df_matches,CONF, LABELS)
display(df_formatted.head(5))
pprint(df_formatted.keys())

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,domination,timedrun,assassination,masterassassination,scavenger
0,2022-04-19 23:03:28,2022-04-19 23:30:56,mp_wz_island,Duos,10717821121770145230,27,1,wz,156,75,...,M19 fists,Sten fists,1,650.0,650.0,1,<NA>,<NA>,<NA>,<NA>
1,2022-04-19 22:48:51,2022-04-19 23:16:20,mp_wz_island,Duos,9298769577395596626,27,1,wz,149,75,...,M19 fists,NaN,2,1075.0,1075.0,1,1,<NA>,<NA>,<NA>
2,2022-04-19 22:26:50,2022-04-19 22:53:23,mp_wz_island,Duos,4690918440001982609,26,1,wz,148,74,...,M19 fists,Sten fists,0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
3,2022-04-19 22:17:18,2022-04-19 22:44:42,mp_wz_island,Duos,9316483071360501762,27,1,wz,151,75,...,M19 fists,NaN,0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
4,2022-04-19 22:08:10,2022-04-19 22:34:36,mp_wz_island,Duos,11643996262360176659,26,1,wz,153,76,...,NaN,NaN,1,500.0,500.0,<NA>,1,<NA>,<NA>,<NA>


Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'version', 'gameType', 'playerCount', 'teamCount', 'draw',
       'privateMatch', 'kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs',
       'score', 'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'percentTimeMoving', 'miscXp',
       'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken', 'team',
       'awards', 'username', 'uno', 'clantag', 'loadout_1', 'loadout_2',
       'loadout_3', 'missionsComplete', 'totalMissionXpEarned',
       'totalMissionWeaponXpEarned', 'domination', 'timedrun', 'assassination',
       'masterassassination', 'scavenger'],
      dtype='object')


#### tests